<a href="https://colab.research.google.com/github/sauravsingla/NLP/blob/master/NLP_Speech_to_Text_Tensor2Tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq tensorflow
!pip install -qq tensor2tensor
!pip install -qq pydub
!apt-get -qq update
!apt-get -qq install -y ffmpeg
!apt-get -qq install -y sox

Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../0-libopencore-amrnb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack .../1-libopencore-amrwb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrwb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libmagic-mgc.
Preparing to unpack .../2-libmagic-mgc_1%3a5.32-2ubuntu0.1_amd64.deb ...
Unpacking libmagic-mgc (1:5.32-2ubuntu0.1) ...
Selecting previously unselected package libmagic1:amd64.
Preparing to unpack .../3-libmagic1_1%3a5.32-2ubuntu0.1_amd64.deb ...
Unpacking libmagic1:amd64 (1:5.32-2ubuntu0.1) ...
Selecting previously unselected package libsox3:amd64.
Preparing to unpack .../4-libsox3_14.4.2-3_amd64.deb ...
Unpacking libsox3:amd64 (14.4.2-3) ...
Selecting previously unselected package libsox-fmt-al

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections
import base64
import cStringIO
import pydub
import shutil
from scipy.io import wavfile

import IPython
import google.colab

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
tfe = tf.contrib.eager
tf.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

# Setup some directories
data_dir = os.path.expanduser("~/t2t/data")
tmp_dir = os.path.expanduser("~/t2t/tmp")
train_dir = os.path.expanduser("~/t2t/train")
checkpoint_dir = os.path.expanduser("~/t2t/checkpoints")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)

gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"

/usr/local/lib/python2.7/dist-packages/scipy/signal/_max_len_seq.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._max_len_seq_inner import _max_len_seq_inner
/usr/local/lib/python2.7/dist-packages/scipy/signal/_upfirdn.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._upfirdn_apply import _output_len, _apply
/usr/local/lib/python2.7/dist-packages/scipy/signal/spectral.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._spectral import _lombscargle
/usr/local/lib/python2.7/dist-packages/scipy/signal/_peak_finding.py:13: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._peak_finding_utils import (_argmaxima1d, _select_by_peak_distance,
/usr/local/lib/python2.7/dist-packages/tensor2tensor/data_generators/video_generated.py:34: UserWarning: 
T

In [ ]:
problem_name = "librispeech_clean"
asr_problem = problems.problem(problem_name)
encoders = asr_problem.feature_encoders(None)

model_name = "transformer"
hparams_set = "transformer_librispeech_tpu"

hparams = trainer_lib.create_hparams(hparams_set,data_dir=data_dir, problem_name=problem_name)
asr_model = registry.model(model_name)(hparams, Modes.PREDICT)

def encode(x):
  waveforms = encoders["waveforms"].encode(x)
  encoded_dict = asr_problem.preprocess_example({"waveforms":waveforms, "targets":[]}, Modes.PREDICT, hparams)
  
  return {"inputs" : tf.expand_dims(encoded_dict["inputs"], 0), "targets" : tf.expand_dims(encoded_dict["targets"], 0)}

def decode(integers):
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["targets"].decode(np.squeeze(integers))


INFO:tensorflow:Unsetting shared_embedding_and_softmax_weights.
:::MLPv0.5.0 transformer 1548939261.586004972 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:59) model_hp_embedding_shared_weights: {"vocab_size": 256, "hidden_size": 384}
INFO:tensorflow:Setting T2TModel mode to 'infer'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0


In [ ]:
# Copy the pretrained checkpoint locally
ckpt_name = "transformer_asr_180214"
gs_ckpt = os.path.join(gs_ckpt_dir, ckpt_name)
print(gs_ckpt)
!gsutil cp -R {gs_ckpt} {checkpoint_dir} 
ckpt_path = tf.train.latest_checkpoint(os.path.join(checkpoint_dir, ckpt_name))
ckpt_path

gs://tensor2tensor-checkpoints/transformer_asr_180214
Copying gs://tensor2tensor-checkpoints/transformer_asr_180214/checkpoint...
Copying gs://tensor2tensor-checkpoints/transformer_asr_180214/model.ckpt-230000.data-00000-of-00001...
Copying gs://tensor2tensor-checkpoints/transformer_asr_180214/model.ckpt-230000.index...
Copying gs://tensor2tensor-checkpoints/transformer_asr_180214/model.ckpt-230000.meta...
\ [4 files][290.3 MiB/290.3 MiB]                                                
Operation completed over 4 objects/290.3 MiB.                                    


u'/root/t2t/checkpoints/transformer_asr_180214/model.ckpt-230000'

In [ ]:
# Restore and transcribe!
def transcribe(inputs):
  encoded_inputs = encode(inputs)
  with tfe.restore_variables_on_create(ckpt_path): 
    model_output = asr_model.infer(encoded_inputs, beam_size=2, alpha=0.6, decode_length=1)["outputs"]
  return decode(model_output)

def play_and_transcribe(inputs):
  waveforms = encoders["waveforms"].encode(inputs)
  IPython.display.display(IPython.display.Audio(data=waveforms, rate=16000))
  return transcribe(inputs) 

In [ ]:
uploaded = google.colab.files.upload()
prerecorded_messages = []

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  mem_file = cStringIO.StringIO(uploaded[fn])
    
  save_filename = os.path.join(tmp_dir, fn)
  with open(save_filename, 'w') as fd:
    mem_file.seek(0)
    shutil.copyfileobj(mem_file, fd)
  prerecorded_messages.append(save_filename)
         
              
for inputs in prerecorded_messages:
  outputs = play_and_transcribe(inputs)

  print("Inputs: %s" % inputs)
  print("Outputs: %s" % outputs)

In [ ]:
# Records webm file and converts
def RecordNewAudioSample(filename=None, webm_filename=None):
  """Args:
          filename - string, path for storing wav file
          webm_filename - string, path for storing webm file
     Returns:
          string - path where wav file was saved. (=filename if specified)
    
  """
  # Create default filenames in tmp_dir if not specified.
  if not filename:
    filename = os.path.join(tmp_dir, "recording.wav")
  if not webm_filename:
    webm_filename = os.path.join(tmp_dir, "recording.webm")
    
  # Record webm file form colab.
  
  audio = google.colab._message.blocking_request('user_media', {"audio":True, "video":False, "duration":-1}, timeout_sec=600)
  #audio = frontend.RecordMedia(True, False)
  
  # Convert the recording into in_memory file.
  music_mem_file = cStringIO.StringIO(
      base64.decodestring(audio[audio.index(',')+1:]))
  
  # Store webm recording in webm_filename. Storing is necessary for conversion.
  with open(webm_filename, 'w') as fd:
    music_mem_file.seek(0)
    shutil.copyfileobj(music_mem_file, fd)
    
  # Open stored file and save it as wav with sample_rate=16000.
  pydub.AudioSegment.from_file(webm_filename, codec="opus"
                              ).set_frame_rate(16000).export(out_f=filename,
                                                             format="wav")
  return filename

In [ ]:
# Record the sample
my_sample_filename = RecordNewAudioSample()
print my_sample_filename

/root/t2t/tmp/recording.wav


In [ ]:
print play_and_transcribe(my_sample_filename)

INFO:tensorflow:Beam Decoding with beam size 2
:::MLPv0.5.0 transformer 1548939344.455209970 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/utils/expert_utils.py:231) model_hp_layer_postprocess_dropout: 0.0
:::MLPv0.5.0 transformer 1548939344.524619102 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:101) model_hp_hidden_layers: 6
:::MLPv0.5.0 transformer 1548939344.782438993 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:101) model_hp_attention_dropout: 0.0
:::MLPv0.5.0 transformer 1548939344.846893072 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:101) model_hp_attention_dense: {"num_heads": 2, "use_bias": "false", "hidden_size": 384}
:::MLPv0.5.0 transformer 1548939346.071619034 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:101) model_hp_norm: {"hidden_size": 384}
:::MLPv0.5.0 transformer 1548939346.458349943 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/